In [16]:
import os
from docx import Document
from docx.shared import Inches
from datetime import datetime


# Bases de datos

In [17]:
DOCUMENTOS_PATH = "documentos/"
FIRMAS_PATH = "firmas/"
MACHOTES_PATH = "machotes/"

In [18]:
base_usuarios = {
    "ana.lopez@casamonarca.org": "Ana López",
    "carlos.ramirez@casamonarca.org": "Carlos Ramírez",
    "maria.gonzalez@casamonarca.org": "María González"
}
# Formato de firmas firmas/maria_dot_gonzalez_at_casamonarca_dot_org.png


# Codigo

In [19]:
def normalizar_correo(correo):
    return correo.strip().lower().replace("@", "_at_").replace(".", "_dot_")

In [20]:
# En local
def buscar_firma_por_correo(correo):
    archivo = normalizar_correo(correo) + ".png"
    path = os.path.join(FIRMAS_PATH, archivo)
    if not os.path.exists(path):
        raise FileNotFoundError(f"Firma no encontrada para {correo}")
    return path
# En produccion base online

# En local
def buscar_nombre_por_correo(correo, base_usuarios):
    return base_usuarios.get(correo, None)

# En produccion pasarlo como parametro o buscarlo en azure

# En local
def obtener_documento_por_id(doc_id):
    path = os.path.join(DOCUMENTOS_PATH, f"doc_{doc_id}.docx")
    if not os.path.exists(path):
        raise FileNotFoundError(f"Documento con ID {doc_id} no encontrado")
    return path

# En produccion pasarlo como parametro o buscarlo en azure




In [21]:
def rellenar_campos_docx(path_machote, datos, id_documento):
    """
    Rellena los campos {campo} en un documento .docx y guarda usando un ID único.

    Args:
        path_machote (str): Ruta del documento machote (.docx) a usar.
        datos (dict): Diccionario con claves {campo} y sus valores correspondientes.
        id_documento (str or int): ID único del documento (ejemplo: 123456)
    """
    # Cargar el documento
    doc = Document(path_machote)

    # Reemplazar en cada párrafo
    for p in doc.paragraphs:
        for key, value in datos.items():
            if key in p.text:
                p.text = p.text.replace(key, str(value))

    # Reemplazar dentro de tablas también
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                for key, value in datos.items():
                    if key in cell.text:
                        cell.text = cell.text.replace(key, str(value))

    # Definir ruta de salida basada en el ID
    path_salida = os.path.join(DOCUMENTOS_PATH, f"doc_{id_documento}.docx")

    # Guardar el documento rellenado en su nueva ruta
    doc.save(path_salida)
    return path_salida

In [22]:
def insertar_firma_y_nombre_por_id(doc_id, correo, base_usuarios, placeholder_nombre, rol):
    """
    Inserta la firma y nombre en un documento buscando por ID, y guarda sobre el mismo archivo.
    """
    doc_path = obtener_documento_por_id(doc_id)
    doc = Document(doc_path)

    nombre = base_usuarios.get(correo)
    if not nombre:
        raise ValueError(f"Nombre no encontrado para {correo}")

    firma_path = buscar_firma_por_correo(correo)

    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                for i, paragraph in enumerate(cell.paragraphs):
                    if placeholder_nombre in paragraph.text:
                        # Insertar firma
                        parrafo_firma = cell.paragraphs[i].insert_paragraph_before()
                        run = parrafo_firma.add_run()
                        run.add_picture(firma_path, width=Inches(2))
                        
                        # Insertar una línea de firma si quieres que siempre esté
                        parrafo_linea = paragraph.insert_paragraph_before()
                        parrafo_linea.add_run("____________________")

                        # Reemplazar marcador por nombre + rol
                        paragraph.text = f"{rol}: {nombre}\n(Personal autorizado)"


    # Guardar editando sobre el mismo documento
    doc.save(doc_path)

In [23]:
def crear_documento_donacion(datos):
    """
    Crea todo el flujo de generación del documento de donación.

    Args:
        path_machote (str): Ruta al machote de especie o dinero.
        datos (dict): Diccionario de datos a rellenar.

    Returns:
        dict: Contiene el ID del documento, tipo de donación y fecha de donación.
    """
    # 1. Generar ID
    id_documento = uuid.uuid4()
    
     # 2. Determinar tipo de donación
    if "{monto_donado}" in datos or "monto_donado" in datos:
        tipo_donacion = "dinero"
        path_machote = os.path.join(MACHOTES_PATH, "Donacion_Dinero.docx")
    elif "{lista_articulos}" in datos or "lista_articulos" in datos:
        tipo_donacion = "especie"
        path_machote = os.path.join(MACHOTES_PATH, "Donacion_Especie.docx")
    else:
        tipo_donacion = "desconocido"  # En caso de error
        
    


    # 3. Determinar la ruta del machote
    path_final = rellenar_campos_docx(path_machote, datos, id_documento)

   

    # 4. Obtener fecha
    fecha_donacion = datos.get("{fecha_donacion}", datos.get("fecha_donacion", "fecha_desconocida"))

    return {
        "id_documento": id_documento,
        "tipo_donacion": tipo_donacion,
        "fecha_donacion": fecha_donacion,
        "path_documento": path_final
    }

In [24]:
today = datetime.now()
fecha_hoy = today.strftime("%d de %B de %Y")
datos_donacion = {
    "{fecha_donacion}": fecha_hoy,
    "{nombre_donante}": "Juan Pérez",
    "{lista_articulos}": "- 10 paquetes de arroz\n- 5 cobijas\n- 3 cajas de medicamento"
}

resultado = crear_documento_donacion(
    datos=datos_donacion
)

print(resultado)


{'id_documento': UUID('8aa79420-7384-4562-b1e1-eaa429a4961c'), 'tipo_donacion': 'especie', 'fecha_donacion': '28 de April de 2025', 'path_documento': 'documentos/doc_8aa79420-7384-4562-b1e1-eaa429a4961c.docx'}


In [ ]:
insertar_firma_y_nombre_por_id(
    doc_id=resultado["id_documento"],
    correo="carlos.ramirez@casamonarca.org",
    base_usuarios=base_usuarios,
    placeholder_nombre="{nombre_inventario}",
    rol="Aprobado por")